# Testing Direct Reduction Strategy

In [42]:
from forecasting_functions import *
from data_processing_functions import *
import lightgbm as lgb
import numpy as np

from sktime.forecasting.model_selection import SlidingWindowSplitter

In [43]:
# import weekly finance time series
# ignore header and skip the first row to use integers as column names
full_data = pd.read_csv("../../Data/Train/Clean/full_m3_monthly_micro_clean.csv", header=None, skiprows=1)

In [44]:
# convert to a list of series, potentially with different lengths
full_data = [x.dropna() for _, x in full_data.iterrows()]

In [45]:
np.unique([len(y) for y in full_data])

array([ 67,  68, 125])

In [46]:
67-18

49

In [47]:
49-18

31

In [4]:
# forecast horizon
h = 1

In [5]:
Y = [x.iloc[:-h] for x in full_data]
Test = [x.iloc[-h:] for x in full_data]
Test = pd.DataFrame([x.reset_index(drop=True) for x in Test]).T

In [6]:
y = np.arange(1,101)

In [7]:
def reduce_train_test(train_data, window_length, target_forecast_period):
    cv = SlidingWindowSplitter(window_length=window_length, fh=target_forecast_period, start_with_window=True)
    X = []
    target = []
    for input_window, output_window in cv.split(train_data):
        X.append(train_data[input_window].reset_index(drop=True))
        target.append(train_data[output_window].reset_index(drop=True))
        
    X = np.array(pd.concat(X, axis=1).T)
    target = np.array(pd.concat(target, axis=0))
    
    return X, target

In [8]:
reduce_train_test(Y[0], 10, 1)

(array([[2640., 2160., 4200., 3360., 2400., 3600., 1920., 4200., 4560.,
          480.],
        [2160., 4200., 3360., 2400., 3600., 1920., 4200., 4560.,  480.,
         3720.],
        [4200., 3360., 2400., 3600., 1920., 4200., 4560.,  480., 3720.,
         5640.],
        [3360., 2400., 3600., 1920., 4200., 4560.,  480., 3720., 5640.,
         2880.],
        [2400., 3600., 1920., 4200., 4560.,  480., 3720., 5640., 2880.,
         1800.],
        [3600., 1920., 4200., 4560.,  480., 3720., 5640., 2880., 1800.,
         3120.],
        [1920., 4200., 4560.,  480., 3720., 5640., 2880., 1800., 3120.,
         2400.],
        [4200., 4560.,  480., 3720., 5640., 2880., 1800., 3120., 2400.,
         2520.],
        [4560.,  480., 3720., 5640., 2880., 1800., 3120., 2400., 2520.,
         9000.],
        [ 480., 3720., 5640., 2880., 1800., 3120., 2400., 2520., 9000.,
         2640.],
        [3720., 5640., 2880., 1800., 3120., 2400., 2520., 9000., 2640.,
         3120.],
        [5640., 2880.

In [9]:
processed, last_window_dt, last_window = reduce_train_test_global(Y, 10)

In [31]:
def reduce_train_test_global_2(train_data, window_length, target_forecast_period):
    # slightly modified code from the M4 competition
    # adapted from code in sktime tutorial.
    """
    Splits the series into train and test sets.

    :param train_data: a 2-d numpy array with series as rows.
    :param window_length: window_length.
    :param h: number of out of sample points (forecast horizon length).
    :return X_train, Y_train: reduced training datasets.
    """

    # store number of time series
    num_series = len(train_data)

    # empty lists for training data
    X_train = []
    Y_train = []

    for series in train_data:
        x_train, y_train = reduce_train_test(series, window_length, target_forecast_period)
        X_train.append(x_train)
        Y_train.append(y_train)
    
    last_x = [x[-1,:] for x in X_train]
    last_y = [y[-1] for y in Y_train]
    
    # testing this line
    last_window = [i.iloc[-window_length:] for i in train_data]

    # last_window = [pd.Series(np.concatenate([last_x[i][1:], np.expand_dims(last_y[i], axis=0)])) for i in range(len(last_x))]

    X_train = np.concatenate(X_train)
    Y_train = np.concatenate(Y_train)

    # concatenate outcome with features
    combined = pd.DataFrame(np.hstack([X_train, Y_train.reshape(X_train.shape[0],1)]))

    # perform per-window trend normalization (detrending) for training data and
    # last window
    detrender = Detrender()
    last_window_dt = [detrender.fit_transform(x) for x in last_window]
    combined = combined.apply(lambda x: detrender.fit_transform(x), axis=1)

    # add correct time index back to last_window
#     for i in range(num_series):
#         last_window[i].index = train_data[i].index[-window_length:]
#         last_window_dt[i].index = train_data[i].index[-window_length:]

    return combined, last_window_dt, last_window

In [38]:
processed2, last_window_dt2, last_window2 = reduce_train_test_global_2(Y, 10, 18)

In [41]:
processed2

,0,1,2,3,4,5,6,7,8,9,10
0,54.545455,-565.090909,1335.272727,355.636364,-744.000000,316.363636,-1503.272727,637.090909,857.454545,-3362.181818,2618.181818
1,-812.727273,1191.272727,315.272727,-680.727273,483.272727,-1232.727273,1011.272727,1335.272727,-2780.727273,423.272727,747.272727
2,1380.000000,377.454545,-745.090909,292.363636,-1550.181818,567.272727,764.727273,-3477.818182,-400.363636,1357.090909,1434.545455
3,207.272727,-748.363636,456.000000,-1219.636364,1064.727273,1429.090909,-2646.545455,597.818182,2522.181818,-233.454545,-1429.090909
4,-169.090909,855.272727,-1000.363636,1104.000000,1288.363636,-2967.272727,97.090909,1841.454545,-1094.181818,-2349.818182,2394.545455
...,...,...,...,...,...,...,...,...,...,...,...
30166,-158.863636,-175.545455,-17.227273,246.090909,174.409091,222.727273,136.045455,-275.636364,-142.318182,-9.000000,-0.681818
30167,-240.227273,-70.181818,204.863636,144.909091,204.954545,130.000000,-269.954545,-124.909091,20.136364,-84.818182,85.227273
30168,-161.363636,128.363636,83.090909,157.818182,97.545455,-287.727273,-128.000000,31.727273,-58.545455,76.181818,60.909091
30169,122.500000,66.909091,131.318182,60.727273,-334.863636,-185.454545,-36.045455,-136.636364,-12.227273,212.181818,111.590909


In [39]:
[np.sum(last_window[i] == last_window2[i]) for i in range(len(last_window))]

[10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,


In [40]:
last_window2

[56    1440.0
 57     240.0
 58    1800.0
 59    4680.0
 60    1800.0
 61    1680.0
 62    3720.0
 63    2160.0
 64     480.0
 65    2040.0
 Name: 0, dtype: float64,
 56    1320.0
 57    1080.0
 58    1680.0
 59    1080.0
 60     840.0
 61     840.0
 62     600.0
 63    1320.0
 64    1320.0
 65    2280.0
 Name: 1, dtype: float64,
 56    2550.0
 57    8640.0
 58    7680.0
 59    6720.0
 60    5190.0
 61    5190.0
 62    5910.0
 63    4800.0
 64    5640.0
 65    5790.0
 Name: 2, dtype: float64,
 56     2280.0
 57    12300.0
 58     8800.0
 59     9660.0
 60     5860.0
 61     6600.0
 62     8100.0
 63     7760.0
 64     6940.0
 65     8020.0
 Name: 3, dtype: float64,
 56     6450.0
 57     9800.0
 58    12450.0
 59    10050.0
 60     8400.0
 61    10000.0
 62    10950.0
 63    10700.0
 64     8800.0
 65     7550.0
 Name: 4, dtype: float64,
 56    2700.0
 57    6150.0
 58    1500.0
 59    1650.0
 60    1650.0
 61    4050.0
 62    1650.0
 63    3150.0
 64     150.0
 65    3900.0
 Name: 5, 

***